# Create, evaluate, and score a churn prediction model

## Introduction

In this notebook series, you'll see a Microsoft Fabric data science workflow with an end-to-end example. The scenario is to build a model to predict whether bank customers would churn or not. The churn rate, also known as the rate of attrition refers to the rate at which bank customers stop doing business with the bank.

The main steps in this notebook series are:

**Notebook 1: Data Ingestion** <br>
&nbsp; &nbsp; 1. Install custom libraries <br>
&nbsp; &nbsp; 2. Load the data <br>

**Notebook 2: Data Preparation** <br>
&nbsp; &nbsp; 3. Understand and process the data through exploratory data analysis and demonstrate the use of Fabric Data Wrangler feature.

**Notebook 3: Model Training**<br>
&nbsp; &nbsp; 4. Train machine learning models using `Scikit-Learn` and `LightGBM`, and track experiments using MLflow and Fabric Autologging feature.<br>
&nbsp; &nbsp; 5. Evaluate and save the final machine learning model.<br>

**Notebook 4: Inference**<br>
&nbsp; &nbsp; 6. load the best model to run predicitons.<br>


## Prerequisites
- [Add a lakehouse](https://aka.ms/fabric/addlakehouse) to this notebook. You will be downloading data from a public blob, then storing the data in the lakehouse. 

## Step 1: Install custom libraries


When developing a machine learning model or doing ad-hoc data analysis, you may need to quickly install a custom library (e.g., `imblearn` in this notebook) for the Apache Spark session. To do this, you have two choices. 

1. You can use the in-line installation capabilities (e.g., `%pip`, `%conda`, etc.) to quickly get started with new libraries. Note that this installation option would install the custom libraries only in the current notebook and not in the workspace.

```python
# Use pip to install libraries
%pip install <library name>

# Use conda to install libraries
%conda install <library name>
 
```
2. Alternatively, you can create a Fabric environment, install libraries from public sources or upload custom libraries to it, and then your workspace admin can attach the environment as the default for the workspace. All the libraries in the environment will then become available for use in any notebooks and Spark job definitions in the workspace. For more information on environments, see [create, configure, and use an environment in Microsoft Fabric](https://aka.ms/fabric/create-environment).

For this notebook, you'll install the `imblearn` using `%pip install`. Note that the PySpark kernel will be restarted after `%pip install`, thus you'll need to install the library before you run any other cells.

In [ ]:
# Use pip to install imblearn for SMOTE
# %pip install imblearn --quiet

## Step 2: Ingest data into a Microsoft Fabric lakehouse

### Bank churn dataset

The dataset in *churn.csv* contains the churn status of 10,000 customers, along with 14 attributes that include:

- Credit score
- Geographical location (Germany, France, Spain)
- Gender (male, female)
- Age
 Tenure (number of years the person was a customer at that bank)
- Account balance
- Estimated salary
- Number of products that a customer purchased through the bank
- Credit card status (whether or not the customer has a credit card)
- Active member status (whether or not the person is an active bank customer)

The dataset also includes row number, customer ID, and customer surname columns. Values in these columns shouldn't influence a customer's decision to leave the bank.

A customer bank account closure event defines the churn for that customer. The dataset `Exited` column refers to the customer's abandonment. Since we have little context about these attributes, we don't need background information about the dataset. We want to understand how these attributes contribute to the `Exited` status.

Out of those 10,000 customers, only 2037 customers (roughly 20%) left the bank. Because of the class imbalance ratio, we recommend generation of synthetic data. Confusion matrix accuracy might not have relevance for imbalanced classification. We might want to measure the accuracy using the Area Under the Precision-Recall Curve (AUPRC).

- This table shows a preview of the `churn.csv` data:

|CustomerID|Surname|CreditScore|Geography|Gender|Age|Tenure|Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
|---|---|---|---|---|---|---|---|---|---|---|---|---|
|15634602|Hargrave|619|France|Female|42|2|0.00|1|1|1|101348.88|1|
|15647311|Hill|608|Spain|Female|41|1|83807.86|1|0|1|112542.58|0|

### Imports and parameters

In [ ]:
import os
import requests
from pathlib import Path, PurePath, PurePosixPath

Define these parameters, so that you can use this notebook with different datasets or [Assign parameters values from a pipeline](https://learn.microsoft.com/en-us/fabric/data-engineering/author-execute-notebook#assign-parameters-values-from-a-pipeline).

In [ ]:
# Specify the storage location for the data set
DATA_ROOT = "/lakehouse/default"
DATA_FOLDER = "Files/churn"  # Folder with data files
DATA_FILE = "churn.csv"  # Data file name
REMOTE_URL = "https://synapseaisolutionsa.blob.core.windows.net/public/bankcustomerchurn" #URL of demo data
OUTPUT_TABLE_NAME = "churn"

### Download dataset and upload to lakehouse

In [ ]:
# With an Azure Synapse Analytics blob, this can be done in one line

# Download demo data files into lakehouse if not exist
file_list = [DATA_FILE]
download_path = PurePosixPath(DATA_ROOT, DATA_FOLDER, 'raw')

if not os.path.exists("/lakehouse/default"):
    raise FileNotFoundError(
        "Default lakehouse not found, please add a lakehouse and restart the session."
    )
os.makedirs(download_path, exist_ok=True)
for fname in file_list:
    if not os.path.exists(Path(download_path, fname)):
        r = requests.get(f"{REMOTE_URL}/{fname}", timeout=30)
        with open(Path(download_path, fname), "wb") as f:
            f.write(r.content)
        print("Downloaded demo data files into lakehouse.")
    else: 
        print("Demo data files already exists.")

### Read raw data from the lakehouse

This code reads raw data from the **Files** section of the lakehouse, and adds more columns for different date parts. Creation of the partitioned delta table uses this information.

In [ ]:
df = (
    spark.read.option("header", True)
    .option("inferSchema", True)
    .csv(str(PurePath(DATA_FOLDER, 'raw', DATA_FILE)))
    .cache()
)

### Create a delta table

In [ ]:
# Create a PySpark DataFrame from pandas
output_table_name = f"bronze/{OUTPUT_TABLE_NAME}"
df.write.mode("overwrite").format("delta").save(f"Tables/{output_table_name}")
print(f"Spark DataFrame saved to delta table: {output_table_name}")